# FHIR-Drills (Simple Patient)

This notebook is a rough, Jupyter translation of the FHIR-Drills tutorial based on Postman avaialble [here](https://fhir-drills.github.io/simple-patient.html). Rather than using the Python Smart on FHIR library, I'm going to use the Python requests library so that we are working more directly with the HTTP protocol and are closer to the steps described in the original tutorial than would be achieved with the FHIR Python client.

For more details, please refer to the original tutorials.

FHIR is built upon web standards, particularly [Representational state transfer (REST)](https://en.wikipedia.org/wiki/Representational_state_transfer). 

In this tutorial we will explore RESTful operations to create, read, update, and delete resources (CRUD) with a FHIR server. 

We will interact with the FHIR server using the Hype Text Transfer Protocol [(HTTP)](https://en.wikipedia.org/wiki/Hypertext_Transfer_Protocol), a digital communications protocol develooped by Tim Berners-Lee at CERN. We will use the Python package [`requests`](https://requests.readthedocs.io/en/master/index.html) to implement the HTTP commands for us. `requests` will generate URLs that tell the remote server what we want to do, receive the responses from the remote servers, and convert them into Python objects we can interact with.

## Find a patient with relevant variables

In [ ]:
import requests
from post_data import create_resource
import pprint
pp = pprint.PrettyPrinter(indent=2)

### Create Patient and get `id`

I've written some Python code to upload a patient to the server behidn the scences, mimicing what the HTML button does on the original tutorial

When we invoke `create_resource()` we upload a JSON file describing a patient. When this is posted (inserted) into the remote server, this patient is assigned a unique identifier which is necessary to interact with the patient data remotely. This identifer is then stored in the variable `patientId`.

In [ ]:
patientId = create_resource()

In [ ]:
patientId

### Set the headers to work with FHIR data

Data transfer with HTTP relies on specifying the [media type (MIME type)](https://en.wikipedia.org/wiki/Hypertext_Transfer_Protocol). This is done with the HTTP header. So we need to specify that we will be using FHIR data. We will also specify what character encoding we will be using.

Finally, we need to know the Uniform Resource Locator (URL) of the FHIR server.

In [ ]:
FHIRJSONMimeType = 'application/fhir+json'
header_defaults = {
            'Accept': FHIRJSONMimeType,
            'Accept-Charset': 'UTF-8',
        }
URL = 'https://stu3.test.pyrohealth.net/fhir'

To request the FHIR resource, we build up a URL based on

- The base URL: `URL`
- The resource type we want: e.g. `Patient`
- The ID of the resoruce we want: e.g. `patientID` (returned from `create_resource()`)

In [ ]:
rurl = URL+ '/Patient/'+patientId
rurl

## Get Our Patient

We will use the HTTP [GET](https://www.w3.org/Protocols/rfc2616/rfc2616-sec9.html) method.

In [ ]:
r = requests.get(rurl,
                 headers = header_defaults)

`r` contains the results of our request. It has multiple parts

- `r.status_code`: Is the code returned by the remote server. `200` means success! I ran across many bad codes trying to get things working.
- `r.headers`: The metadata about the request and response
- `r.text`: the raw "stuff" returned
- `r.json`: A method to convert the data returned by the server into a Python dictionary.

In [ ]:
print(r.status_code)
print(r.headers)

In [ ]:
print(r.text)

In [ ]:
pp.pprint(r.json())

## Get the returned content

We are going to extract the returned data and use the resposne objects `json` method to convert the returned object to a Python dictionary.

In [ ]:
patient = r.json()

In [ ]:
pp.pprint(patient)

Dictionary are __key__/__value__ pairs.

We can list the keys with the `keys()` method.

In [ ]:
patient.keys()

We access the values in the dictionary using square brackets and the keys.

For example,

In [ ]:
patient["birthDate"]

We can change the value by assigning a value to the key

In [ ]:
patient["birthDate"] = "1970-01-01"

## Change Patient name

The tutorial suggests we change the patient name to your own and then push the results up to the server.

#### To Do: Modify the values below to match your name rather than mine

In [ ]:
patient["name"] = [{'family': "Chapman", 'given':['Brian', 'Earl'], 
              "prefix":"Dr.", "text":"Brian Chapman", "use":"official"}]

## PUT the modified data on the server

we will use the `PUT` method to push our modified patient to the server.

In [ ]:
r2 = requests.put(URL+'/Patient/'+patientId,
                 headers = header_defaults,
                 json=patient)
r2.ok

`r2.ok` is a boolean value that basically tells you whether things went as planned.

## Verify Results

In [ ]:
requests.get(URL+'/Patient/'+patientId,
                 headers = header_defaults).json()['name']

### Data is Modified!

At least for me!

## Delete the Resource

To delete we simply use the delete method and provide the URL of the resource we want to delete.

In [ ]:
r3 = requests.delete(URL+'/Patient/'+patientId)


In [ ]:
print(r3.status_code)
print(r3.ok)

## Verify the patient is deleted.

In [ ]:
rd = requests.get(URL+'/Patient/'+patientId,
                 headers = header_defaults)

In [ ]:
rd.status_code

status code `410`  means `Gone`

## The data history is preserved!

In [ ]:
versionid = 1
while True:
    print("VERSION: ", versionid)
    rtmp = requests.get(URL+'/Patient/'+patientId+"/_history/%d"%versionid,
                 headers = header_defaults)
    print(rtmp.status_code)
    try:
        pp.pprint(rtmp.json())
        print("-"*42)
        versionid += 1
    except:
        break
    